## 13. 얼굴 인식

In [2]:
import cv2

HWANG = "../images/hwang.jpg"
QUEEN = "../images/queen.jpg"
BEATLES = "../images/beatles.jpg"
SANDVICH = "../images/sandvich.jpg"

face_cascade = cv2.CascadeClassifier("../cascade/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("../cascade/haarcascade_eye.xml")

In [43]:
img = cv2.imread(HWANG)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 색의 밝기로 검출하므로 gray

#scaleFactor - 1.1에 가까울 수록 더 얼굴 정확하게, minNeighbors - 최소 몇개는 얼굴로 잡혀야 얼굴로 인식
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=8, minSize=(10,10))
eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=2, maxSize=(40,40))

if len(faces) :
    for face in faces :
        x, y, width, height = face
        cv2.rectangle(img, (x,y), (x+width, y+height), (0,255,0), 4)

if len(eyes) :
    for eye in eyes :
        x, y, width, height = eye
        cv2.rectangle(img, (x,y), (x+width, y+height), (255,255, 0), 3)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 실습1. 캠화면에 적용
1. 캠화면에서 얼굴과 눈을 찾아서 얼굴 및 눈 영역에 사각형 표시
2. 네모 위에 Face, Eye라는 글자 띄우기
- 추가: 신뢰도를 보여주는 weights를 화면에도 띄울 수 있도록

In [5]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("../cascade/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("../cascade/haarcascade_eye.xml")

while cap.isOpened() : # 캠이 켜져 있는 동안
    ret, frame = cap.read()

    if not ret :
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces, weights_face = face_cascade.detectMultiScale2(gray, scaleFactor=1.1, minNeighbors=4, minSize=(200,200)) # 각 프레임에 대하여 안면인식
    eyes, weights_eye = eye_cascade.detectMultiScale2(gray, scaleFactor=1.1, minNeighbors=4, maxSize=(40,40)) # 눈 인식

    if len(faces) : # 안면인식이 되었다면
        for x, y, w, h in faces:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"Face: {weights_face[0]}", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 2) # weights - 신뢰도

    if len(eyes) : # 눈인식이 되었다면
        for i, eye in enumerate(eyes): # 눈의 숫자와 눈 정보
            x, y, w, h = eye # 눈 정보 따로 분해
            cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,0), 2)
            cv2.putText(frame, f"Eyes : {weights_eye[i]}", (x,y - 5), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 2)

    cv2.imshow("video", frame) # 프레임 지속적으로 출력

    if cv2.waitKey(1) == ord("q") : 
        break

cap.release()
cv2.destroyAllWindows()

### 실습2. 눈 덮어씌우기
1. 원하는 사람 이미지에서 눈을 찾아서
2. 해당 영역에 누늘 덮어 씌워보기
3. 완료시 영상으로 도전

In [15]:
LEFT_EYE = "../images/left_eye.jpg"
RIGHT_EYE = "../images/right_eye.jpg"
eye_cascade = cv2.CascadeClassifier("../cascade/haarcascade_eye.xml")

img = cv2.imread(HWANG)

#오른쪽, 왼쪽눈 구하기
l_eye = cv2.imread(LEFT_EYE)
l_eye = cv2.pyrDown(l_eye)
l_w, l_h, l_channel = l_eye.shape

r_eye = cv2.imread(RIGHT_EYE)
r_eye = cv2.pyrDown(r_eye)
r_w, r_h, r_channel = r_eye.shape

#사진에서 눈 영역 찾기
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, maxSize=(40, 40))

# 각 눈마다 눈 이미지 삽입
if len(eyes):
    for i, eye in enumerate(eyes):
        x, y, width, height = eye
        if i % 2 == 0: # 왼쪽 눈
            img[y : y + l_h, x : x + l_w] = l_eye
        else: # 오른쪽 눈
            img[y : y + r_h, x : x + r_w] = r_eye
        
        #cv2.rectangle(img, (x,y), (x + w, y + h), (255, 255, 0), 2)
        

cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 캠화면 사각형에 눈 사이즈 맞춘 경우
cap = cv2.VideoCapture(0)
eye_cascade = cv2.CascadeClassifier("../cascade/haarcascade_eye.xml")
LEFT_EYE = "../images/left_eye.jpg"
RIGHT_EYE = "../images/right_eye.jpg"

#오른쪽, 왼쪽눈 구하기
l_eye = cv2.imread(LEFT_EYE)
l_w, l_h, l_channel = l_eye.shape

r_eye = cv2.imread(RIGHT_EYE)
r_w, r_h, r_channel = r_eye.shape

while cap.isOpened() : # 캠이 켜져 있는 동안
    ret, frame = cap.read()

    if not ret :
        break
    
    #눈 인식
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, maxSize=(40,40))

    # 각 눈마다 눈 이미지 삽입
    if len(eyes):
        for i, eye in enumerate(eyes): # n번째, 눈
            x, y, width, height = eye
            if i % 2 == 0: # 왼쪽 눈
                l_eye = cv2.resize(l_eye, (width, height))
                frame[y : y + height, x : x + width] = l_eye
            else: # 오른쪽 눈
                r_eye = cv2.resize(l_eye, (width, height))
                frame[y : y + width, x : x + height] = r_eye

    cv2.imshow("video", frame) # 프레임 지속적으로 출력

    if cv2.waitKey(1) == ord("q") : 
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# 눈 사이즈를 그냥 한번 줄이기만 한 경우
cap = cv2.VideoCapture(0)
eye_cascade = cv2.CascadeClassifier("../cascade/haarcascade_eye.xml")
LEFT_EYE = "../images/left_eye.jpg"
RIGHT_EYE = "../images/right_eye.jpg"

#오른쪽, 왼쪽눈 구하기
l_eye = cv2.imread(LEFT_EYE)
l_eye = cv2.pyrDown(l_eye)
l_w, l_h, l_channel = l_eye.shape

r_eye = cv2.imread(RIGHT_EYE)
r_eye = cv2.pyrDown(r_eye)
r_w, r_h, r_channel = r_eye.shape

while cap.isOpened() : # 캠이 켜져 있는 동안
    ret, frame = cap.read()

    if not ret :
        break
    
    #눈 인식
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, maxSize=(40,40))

    # 각 눈마다 눈 이미지 삽입
    if len(eyes):
        for i, eye in enumerate(eyes): # n번째, 눈
            x, y, width, height = eye
            if i % 2 == 0: # 왼쪽 눈
                frame[y : y + l_h, x : x + l_w] = l_eye
            else: # 오른쪽 눈
                frame[y : y + r_h, x : x + r_w] = r_eye

    cv2.imshow("video", frame) # 프레임 지속적으로 출력

    if cv2.waitKey(1) == ord("q") : 
        break

cap.release()
cv2.destroyAllWindows()